### Homework 6

#### 1. Import libraries and read data

In [2]:
import altair as alt
import numpy as np
import pandas as pd
import geopandas as gpd

In [3]:
population_df = pd.read_csv('population_trends.csv')
population_df

,region,year,rate
0,Ukraine,1989,1.7
1,Ukraine,1990,0.5
2,Ukraine,1991,-0.8
3,Ukraine,1992,-2.0
4,Ukraine,1993,-3.5
...,...,...,...
863,Sevastopol',2015,NaN
864,Sevastopol',2016,NaN
865,Sevastopol',2017,NaN
866,Sevastopol',2018,NaN


In [4]:
ukraine_df = gpd.read_file('ukraine.json')
ukraine_df.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,UKR,Ukraine,UKR.1_1,Cherkasy,Cherkas'ka Oblast'|Cherkasskaya Oblast'|Cherkassy,None,Oblast',Region,None,UA.CK,"MULTIPOLYGON (((31.32614 48.74507, 31.31716 48..."
1,UKR,Ukraine,UKR.2_1,Chernihiv,Chernigov|Tschernigow,None,Oblast',Region,None,UA.CH,"MULTIPOLYGON (((33.09283 50.50966, 33.09261 50..."
2,UKR,Ukraine,UKR.3_1,Chernivtsi,Chernivets'ka Oblast'|Chernovitskaya Oblast'|C...,None,Oblast',Region,None,UA.CV,"MULTIPOLYGON (((24.93280 47.72794, 24.93301 47..."
3,UKR,Ukraine,UKR.4_1,Crimea,Crimée|Criméia|Krim|Krymskaya Respublika|Respu...,None,Autonomous Republic,Autonomous Republic,None,UA.KR,"MULTIPOLYGON (((33.79291 44.39153, 33.79465 44..."
4,UKR,Ukraine,UKR.5_1,Dnipropetrovs'k,Dnipropetrovsk|Dniepropietrovsk|Dnjepropetrowsk,None,Oblast',Region,None,UA.DP,"MULTIPOLYGON (((33.93176 47.48407, 33.92332 47..."


#### 2. Створити інтерактивн візуалізацію данихна основі графіків у ДЗ 5

In [5]:
region_selection =  alt.selection_single(on = 'mouseover', nearest = False, empty = 'all', fields = ['region'])

In [6]:
df_2019 = population_df[(population_df["year"] == 2019)]
ukraine_df.rename(columns={'NAME_1':'region'}, inplace=True)
df1 = ukraine_df.merge(df_2019)
df1.head()

,GID_0,NAME_0,GID_1,region,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,year,rate
0,UKR,Ukraine,UKR.1_1,Cherkasy,Cherkas'ka Oblast'|Cherkasskaya Oblast'|Cherkassy,None,Oblast',Region,None,UA.CK,"MULTIPOLYGON (((31.32614 48.74507, 31.31716 48...",2019,-10.0
1,UKR,Ukraine,UKR.2_1,Chernihiv,Chernigov|Tschernigow,None,Oblast',Region,None,UA.CH,"MULTIPOLYGON (((33.09283 50.50966, 33.09261 50...",2019,-12.5
2,UKR,Ukraine,UKR.3_1,Chernivtsi,Chernivets'ka Oblast'|Chernovitskaya Oblast'|C...,None,Oblast',Region,None,UA.CV,"MULTIPOLYGON (((24.93280 47.72794, 24.93301 47...",2019,-3.1
3,UKR,Ukraine,UKR.4_1,Crimea,Crimée|Criméia|Krim|Krymskaya Respublika|Respu...,None,Autonomous Republic,Autonomous Republic,None,UA.KR,"MULTIPOLYGON (((33.79291 44.39153, 33.79465 44...",2019,NaN
4,UKR,Ukraine,UKR.5_1,Dnipropetrovs'k,Dnipropetrovsk|Dniepropietrovsk|Dnjepropetrowsk,None,Oblast',Region,None,UA.DP,"MULTIPOLYGON (((33.93176 47.48407, 33.92332 47...",2019,-8.9


In [14]:
map1 = alt.Chart(ukraine_df).mark_geoshape(fill = 'lightgray', stroke = 'white', strokeWidth = 1)
map2 = alt.Chart(df1).mark_geoshape(stroke = 'white').encode(color = alt.Color('rate:Q',
                                                       scale = alt.Scale(scheme = 'reds',
                                                                         reverse=True),
                                                       title='Rate',
                                                       legend=alt.Legend(orient='bottom-left',
                                                                         tickCount=7,
                                                                         gradientLength = 200)),
                                                             tooltip = [alt.Tooltip('region:N'),
                                                                        alt.Tooltip('rate:N')],
                                                             opacity = alt.condition(region_selection,
                                                                                 alt.value(1),
                                                                                 alt.value(0.3)))

In [15]:
linear_vis = alt.Chart(population_df).mark_line().encode(x = alt.X('year:Q', 
                                                                   scale=alt.Scale(domain=[1988, 2020], nice=False),
                                                                   axis = alt.Axis(format = "1000f")),
                                                         y = alt.Y('rate:Q'),
                                                         detail = alt.Detail('region:N'),
                                                         color=alt.value('grey'),
                                                         tooltip = [alt.Tooltip('region:N'),
                                                                    alt.Tooltip('year:N'),
                                                                    alt.Tooltip('rate:N')],
                                                         opacity = alt.condition(region_selection,
                                                                                 alt.value(1),
                                                                                 alt.value(0.3))) 


In [ ]:
#combine two visualizaions into one 
alt.hconcat((map1 + map2).add_selection(region_selection).properties(width = 900, height = 650),
            linear_vis.add_selection(region_selection).properties(width = 900, height = 650)
           ).properties(background = '#F9F9F9', padding = 25)